In [1]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime
import subprocess
import sys # for args

# THIRD PARTY IMPORTS
import praw

# PROJECT IMPORTS
from setup import reddit
from settings import *
from tools import *
from inbox import check_messages
from reminders import send_reminders
from log_missed_comment import manually_award_comment

subreddit = reddit.subreddit(TARGET_SUBREDDIT)

Loaded 12 users who have opted out from follow-ups.
Loaded 13 users who have opted out from reminders.


Version 7.5.0 of praw is outdated. Version 7.7.1 was released Tuesday July 11, 2023.


# Step 1: Get the last awarded post

In [2]:
# get the last awarded post. We don't need to look back past this time.
last_award = next( reddit.user.me().comments.new() )

In [3]:
print( last_award.permalink )

/r/photocritique/comments/1bbe4jq/how_do_i_edit_this_photo/kukq1mp/


# Step 2: Search the most recent 1,000 comments, and record their IDs

In [4]:
num_processed_comments = 0
num_awarded_comments   = 0

# we can only go back 1,000 comments :|
for awarding_comment in subreddit.comments(limit=None):
    num_processed_comments += 1
    
    if (awarding_comment.author is not None and awarding_comment.author.name != 'AutoModerator') and comment_contains_token( awarding_comment ):
        print(f'Should we award points based on comment {awarding_comment.id} by u/{awarding_comment.author.name}?')
        num_awarded_comments += 1

    if awarding_comment.created_utc <= last_award.created_utc:
        # we've gone back far enough
        print('Yay! There were less than 1,000 comments since the bot crashed.\n')
        break
        
print(f'\n\nSearched {num_processed_comments} comments. Found {num_awarded_comments} that might need awarding.')
        
print(f'There is a gap of {( awarding_comment.created_utc - last_award.created_utc ) / 60 / 60:.1f} hours between the last comment we searched, and the last award.')
print('Hopefully the above number is 0 or small!')

Should we award points based on comment kutue9k by u/MoonIsUpNotDown?
Should we award points based on comment kutnxlx by u/MoonIsUpNotDown?
Should we award points based on comment kut860e by u/rjay9?
Should we award points based on comment kut83ch by u/rjay9?
Should we award points based on comment kut7usv by u/rjay9?
Should we award points based on comment kurmknf by u/Cuckoo_for_Cocaine?
Should we award points based on comment kuqita7 by u/MoonIsUpNotDown?
Should we award points based on comment kuplyc9 by u/MoonIsUpNotDown?
Should we award points based on comment kuplxbz by u/MoonIsUpNotDown?
Should we award points based on comment kuplvdy by u/MoonIsUpNotDown?
Should we award points based on comment kuplu9j by u/MoonIsUpNotDown?
Should we award points based on comment kupltl6 by u/MoonIsUpNotDown?
Should we award points based on comment kuplsni by u/MoonIsUpNotDown?
Should we award points based on comment kuplrw4 by u/MoonIsUpNotDown?
Should we award points based on comment kupkqky

# Step 3: Manually award all the comments above.

Reddit will rate limit you at some point, so you'll need to wait a few mins if you have more than ~15 comments to award.

To award comments, copy the ID above into the below string, and run the cell. Repeat for all comments listed above. You may want to manually check things like:

- don't award to top level comments? 

Are these things still in `run.py`?

In [5]:
from time import sleep

In [8]:
missing_awards = [
# 'kutue9k',
# 'kutnxlx',
# 'kut860e',
# 'kut83ch',
# 'kut7usv',
# 'kurmknf',
# 'kuqita7',
# 'kuplyc9',
# 'kuplxbz',
# 'kuplvdy',
'kuplu9j',
'kupltl6',
'kuplsni',
'kuplrw4',
'kupkqky',
'kupkor1',
'kupkkyl',
'kupkhv7',
'kupkgpe',
'kupkbd7',
'kupk9zt',
'kupk8bl',
'kupk79k',
'kupk5dq',
'kupk3ud',
'kupk1nm',
'kupjzpd',
'kupjybk',
'kup0oaq',
]

In [9]:
for award_id in missing_awards:
    manually_award_comment(award_id)
    sleep(30)

Manually awarding points based on comment kuplu9j by u/MoonIsUpNotDown.
Manually awarding points based on comment kupltl6 by u/MoonIsUpNotDown.
Manually awarding points based on comment kuplsni by u/MoonIsUpNotDown.
Manually awarding points based on comment kuplrw4 by u/MoonIsUpNotDown.
Manually awarding points based on comment kupkqky by u/MoonIsUpNotDown.
Manually awarding points based on comment kupkor1 by u/MoonIsUpNotDown.
Manually awarding points based on comment kupkkyl by u/MoonIsUpNotDown.
Manually awarding points based on comment kupkhv7 by u/MoonIsUpNotDown.
Manually awarding points based on comment kupkgpe by u/MoonIsUpNotDown.
Manually awarding points based on comment kupkbd7 by u/MoonIsUpNotDown.
Manually awarding points based on comment kupk9zt by u/MoonIsUpNotDown.
Manually awarding points based on comment kupk8bl by u/MoonIsUpNotDown.
Manually awarding points based on comment kupk79k by u/MoonIsUpNotDown.
Manually awarding points based on comment kupk5dq by u/MoonIsUpN

# Step 5: Manually look for missed awards that need checking

I would do this by starting at /new, scrolling back to post from the below last-checked comment, and then manually opening all the posts after that and searching for `!CritiquePoint`. Do this until you are fairly certain you've gone far enough.

In [15]:
print('This is the last comment we were able to check:')
print(awarding_comment.permalink)

print('\nComments older than this may need checking manually.')

This is the last comment we were able to check:
/r/photocritique/comments/xza4r1/feedback_request_on_portrait/irmugrd/

Comments older than this may need checking manually.


In [42]:
#  manually_award_comment('irah590')

Manually awarding points based on comment irah590 by u/Koosen-.


In [ ]:
# https://old.reddit.com/r/photocritique/comments/xwqlb5/my_thought_was_better_than_the_execution/irah590/